<a href="https://colab.research.google.com/github/zahir2498/colab-tools/blob/main/2002_Voter_search_Champdani_181.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# @title #🗳️ 2002 Voter Roll Search Tool For Champdani 181 Use Bangla name for best Result
# @markdown #⬅️ Click ▶ the button to start the app.
# @markdown Disclaimer: This tool uses a copied text version of the official voter list PDF.For any mismatches or verification, please check the original PDF available on the official Election Commission website
import re, pandas as pd, os, requests
from IPython.display import display, clear_output, HTML, Javascript
import ipywidgets as widgets

# --- Hide the code cell ---
display(Javascript('''
var code_cells = document.querySelectorAll('.code-cell');
for (var i = 0; i < code_cells.length; i++) {
  code_cells[i].style.display = 'none';
}
'''))

# --- Download the voter roll file automatically from GitHub ---
GITHUB_FILE_URL = "https://github.com/zahir2498/jscss/raw/refs/heads/main/V.txt"
text_file_path = "/content/V.txt"

def ensure_file_downloaded():
    if not os.path.exists(text_file_path):
        print("⬇️ Downloading voter roll file from GitHub...")
        try:
            response = requests.get(GITHUB_FILE_URL)
            if response.status_code == 200:
                with open(text_file_path, 'wb') as f:
                    f.write(response.content)
                print("✅ File downloaded successfully.")
            else:
                print(f"⚠️ Failed to download file. HTTP {response.status_code}")
        except Exception as e:
            print("❌ Error downloading file:", e)

# --- Install Google Translate API if missing ---
try:
    from googletrans import Translator
except:
    print("📦 Installing googletrans... please wait...")
    os.system('pip install googletrans==4.0.0-rc1 > /dev/null')
    from googletrans import Translator

translator = Translator()

# --- Simple Phonetic Converter ---
def eng_to_bangla_phonetic(text):
    mapping = {
        "sh": "শ", "ss": "ষ", "s": "স", "kh": "খ", "k": "ক", "ch": "চ", "c": "চ",
        "t": "ত", "th": "থ", "tt": "ট", "d": "দ", "dh": "ধ", "dd": "ড",
        "n": "ন", "nn": "ণ", "r": "র", "l": "ল", "b": "ব", "bh": "ভ",
        "g": "গ", "gh": "ঘ", "m": "ম", "y": "য", "j": "জ", "jh": "ঝ",
        "h": "হ", "a": "া", "i": "ি", "ee": "ী", "u": "ু", "oo": "ূ",
        "e": "ে", "o": "ো", "oi": "ৈ", "ou": "ৌ"
    }
    text = text.lower()
    for en, bn in mapping.items():
        text = text.replace(en, bn)
    return text

# --- Google Translate helper ---
def eng_to_bangla_google(text):
    if not text.strip():
        return ""
    try:
        translated = translator.translate(text, src='en', dest='bn')
        return translated.text
    except Exception:
        return ""

# --- Combine Google + Phonetic Transliteration ---
def hybrid_translate(text):
    """Use Google Translate + Phonetic transliteration"""
    if not text.strip():
        return ""
    google_text = eng_to_bangla_google(text)
    phonetic_text = eng_to_bangla_phonetic(text)
    if not google_text or re.search(r'[A-Za-z]', google_text):
        return phonetic_text
    return google_text + " " + phonetic_text

# --- Load Text File ---
def load_text():
    ensure_file_downloaded()
    try:
        with open(text_file_path, 'r', encoding='utf-8') as f:
            return f.readlines()
    except FileNotFoundError:
        display(HTML("<b style='color:red;'>⚠️ Could not load voter roll file.</b>"))
        return []

lines = []
search_results = []
partial_results = []

# --- Widgets ---
name_input = widgets.Text(description='নাম / Name:', layout=widgets.Layout(width='400px'))
father_input = widgets.Text(description='পিতা/মাতা:', layout=widgets.Layout(width='400px'))
id_input = widgets.Text(description='ID No:', layout=widgets.Layout(width='400px'))
search_button = widgets.Button(description="🔍 Search", button_style='primary', layout=widgets.Layout(width='150px'))
download_button = widgets.Button(description="⬇️ Download CSV", button_style='success', layout=widgets.Layout(width='200px'))
output_box = widgets.Output()

# --- Search Function ---
def search_voter(b):
    global search_results, partial_results, lines
    with output_box:
        clear_output()
        print("🌐 Preparing search engine (Google + Phonetic)...")

        if not lines:
            lines = load_text()
            if not lines:
                return

        name_q = name_input.value.strip()
        father_q = father_input.value.strip()
        id_q = id_input.value.strip()

        if not any([name_q, father_q, id_q]):
            display(HTML("<b style='color:orange;'>⚠️ Please enter at least one field to search.</b>"))
            return

        # Use hybrid transliteration for English input
        bangla_name = hybrid_translate(name_q) if re.search(r'[A-Za-z]', name_q) else name_q
        bangla_father = hybrid_translate(father_q) if re.search(r'[A-Za-z]', father_q) else father_q

        search_results, partial_results = [], []
        current_part = None

        for line in lines:
            line = line.strip()
            if re.search(r'Part\s*\d+', line, re.IGNORECASE) or re.search(r'অংশ নং\s*\d+', line):
                current_part = line
            if not line or line.startswith(('ক্রমিক', 'Page', 'START', 'নির্বাচক')):
                continue

            exact_name = bangla_name.strip() in line or name_q.strip() in line
            exact_father = bangla_father.strip() in line or father_q.strip() in line
            id_match = id_q.strip() in line if id_q else True

            partial_name = any(word in line for word in bangla_name.split()) or name_q in line
            partial_father = any(word in line for word in bangla_father.split()) or father_q in line

            serial_match = re.match(r"(\d+)", line)
            serial_no = serial_match.group(1) if serial_match else "?"
            id_found = re.findall(r"\b[\w/]{8,}\b", line)
            id_no = ", ".join(id_found) if id_found else ""

            record = {
                "Part No": current_part,
                "Serial No": serial_no,
                "Line": line,
                "ID No": id_no
            }

            if exact_name and exact_father and id_match:
                search_results.append(record)
            elif partial_name and partial_father and id_match:
                partial_results.append(record)

        clear_output()

        if search_results or partial_results:
            if search_results:
                display(HTML("<h4>🔹 Exact Matches</h4>"))
                df_exact = pd.DataFrame(search_results)
                display(df_exact)
            if partial_results:
                display(HTML("<h4>🔸 Partial Matches</h4>"))
                df_partial = pd.DataFrame(partial_results)
                display(df_partial)

            total = len(search_results) + len(partial_results)
            display(HTML(f"<b style='color:green;'>✅ Found {total} records ({len(search_results)} exact, {len(partial_results)} partial).</b>"))
            display(download_button)
        else:
            display(HTML("<b style='color:red;'>❌ No matches found.</b>"))

# --- CSV Download ---
def download_csv(b):
    if not (search_results or partial_results):
        with output_box:
            display(HTML("<b style='color:orange;'>⚠️ No search results to download.</b>"))
        return
    df_all = pd.concat([pd.DataFrame(search_results), pd.DataFrame(partial_results)], ignore_index=True)
    path = "/content/voter_search_results.csv"
    df_all.to_csv(path, index=False, encoding='utf-8-sig')
    with output_box:
        display(HTML(f"<b>✅ CSV ready:</b> <a href='voter_search_results.csv' target='_blank'>Click here to download</a>"))

# --- Bind Buttons ---
search_button.on_click(search_voter)
download_button.on_click(download_csv)

# --- Display UI ---
display(widgets.VBox([
    widgets.HTML("<h3>🗳️ Bangla Voter Roll Search Tool use bangla for best result</h3>"),
    name_input,
    father_input,
    id_input,
    search_button,
    output_box
]))


<IPython.core.display.Javascript object>